In [3]:
!pip install transformers

In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
train = np.load('data.npz')
df= pd.DataFrame.from_dict({item: train[item] for item in train.files})

train_data, val_data = train_test_split(df, test_size=0.2)

test = np.load('test.npz')
test_data= pd.DataFrame.from_dict({item: test[item] for item in test.files})

In [6]:
train_data.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train_data['DATA_COLUMN'] = train_data['DATA_COLUMN']
train_data.head()

,DATA_COLUMN,LABEL_COLUMN
24266,Whilst reading through the comments left for t...,0.0
7291,When I saw the elaborate DVD box for this and ...,0.0
7272,"So, finally I know it exists. Along with the o...",0.0
10539,Legend of Zu<br /><br />I remember well Tsui H...,0.0
22357,"In ""Anne of Green Gables"" (1934), Marilla Cuth...",0.0


In [7]:
test_data.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test_data['DATA_COLUMN'] = test_data['DATA_COLUMN']
test_data.head()

,DATA_COLUMN,LABEL_COLUMN
0,"I found this movie really hard to sit through,...",1.0
1,The movie starts off with Reeve (Ekin) and his...,1.0
2,I had a VERY hard time sitting through this fi...,1.0
3,"I'm not a big fan of musicals, but I was alway...",0.0
4,I honestly fail to understand why people love ...,1.0


In [8]:
val_data.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
val_data['DATA_COLUMN'] = val_data['DATA_COLUMN']
val_data.head()

,DATA_COLUMN,LABEL_COLUMN
19347,It's been said several times - not least by me...,1.0
12140,"This movie surprised me. Some things were ""cli...",0.0
200,This movie was hysterical. I haven't laughed t...,0.0
2747,Please see also my comment on Die Nibelungen p...,0.0
7868,The wit and pace and three show stopping Busby...,0.0


In [9]:
def float2num(value):
    if value=='1.0': 
        return 1
    else: 
        return 0

test_data['LABEL_COLUMN'] = test_data['LABEL_COLUMN'].apply(float2num)
train_data['LABEL_COLUMN'] = train_data['LABEL_COLUMN'].apply(float2num)
val_data['LABEL_COLUMN'] = val_data['LABEL_COLUMN'].apply(float2num)

In [10]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train_data.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = val_data.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [11]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

train_InputExamples, validation_InputExamples = convert_data_to_examples(train_data, 
                                                                           val_data, 
                                                                           DATA_COLUMN, 
                                                                           LABEL_COLUMN)

In [12]:
model_tiny = TFBertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2",from_pt=True)
tokenizer_tiny = BertTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
model_mini = TFBertForSequenceClassification.from_pretrained("google/bert_uncased_L-4_H-256_A-4",from_pt=True)
tokenizer_mini = BertTokenizer.from_pretrained("google/bert_uncased_L-4_H-256_A-4")
model_small = TFBertForSequenceClassification.from_pretrained("google/bert_uncased_L-4_H-512_A-8",from_pt=True)
tokenizer_small = BertTokenizer.from_pretrained("google/bert_uncased_L-4_H-512_A-8")
model_medium = TFBertForSequenceClassification.from_pretrained("google/bert_uncased_L-8_H-512_A-8",from_pt=True)
tokenizer_medium = BertTokenizer.from_pretrained("google/bert_uncased_L-8_H-512_A-8")

Downloading:   0%|          | 0.00/382 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [13]:
model_tiny.summary()
model_mini.summary()
model_small.summary()
model_medium.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  4385920   
                                                                 
 dropout_7 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  258       
                                                                 
Total params: 4,386,178
Trainable params: 4,386,178
Non-trainable params: 0
_________________________________________________________________
Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  11170560  
                                        

In [14]:
training_data_tiny = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer_tiny)
training_data_tiny = training_data_tiny.shuffle(100).batch(32).repeat(2)

validation_data_tiny = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer_tiny)
validation_data_tiny = validation_data_tiny.batch(32)

training_data_mini = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer_mini)
training_data_mini = training_data_mini.shuffle(100).batch(32).repeat(2)

validation_data_mini = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer_mini)
validation_data_mini = validation_data_mini.batch(32)

training_data_small = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer_small)
training_data_small = training_data_small.shuffle(100).batch(32).repeat(2)

validation_data_small = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer_small)
validation_data_small = validation_data_small.batch(32)

training_data_medium = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer_medium)
training_data_medium = training_data_medium.shuffle(100).batch(32).repeat(2)

validation_data_medium = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer_medium)
validation_data_medium = validation_data_medium.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [15]:
print("Training tiny model")
model_tiny.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model_tiny.fit(training_data_tiny, epochs=1, validation_data=validation_data_tiny)

print("Training mini model")
model_mini.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model_mini.fit(training_data_mini, epochs=1, validation_data=validation_data_mini)

print("Training small model")
model_small.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model_small.fit(training_data_small, epochs=1, validation_data=validation_data_small)

print("Training medium model")
model_medium.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model_medium.fit(training_data_medium, epochs=1, validation_data=validation_data_medium)

Training tiny model
1250/1250 [==============================] - 102s 73ms/step - loss: 0.5083 - accuracy: 0.7490 - val_loss: 0.4109 - val_accuracy: 0.8138
Training mini model
1250/1250 [==============================] - 277s 214ms/step - loss: 0.4171 - accuracy: 0.8066 - val_loss: 0.3580 - val_accuracy: 0.8382
Training small model
1250/1250 [==============================] - 575s 452ms/step - loss: 0.3619 - accuracy: 0.8367 - val_loss: 0.3442 - val_accuracy: 0.8524
Training medium model
1250/1250 [==============================] - 1093s 861ms/step - loss: 0.3287 - accuracy: 0.8565 - val_loss: 0.3350 - val_accuracy: 0.8602


In [29]:

def batching(tokenizer,model,):
    test_data_list=[]
    test_data_list=np.array_split(test_data,500)
    label_list =[]
    i=0
    for i in range(len(test_data_list)):
        test_data_list[i]=test_data_list[i]['DATA_COLUMN'].values.tolist()
        tf_batch = tokenizer(test_data_list[i], max_length=128, padding=True, truncation=True, return_tensors='tf')
        tf_outputs = model(tf_batch)
        tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
        label = tf.argmax(tf_predictions, axis=1)
        label = label.numpy()
        label_list.append(label)
    return label_list

batch_tiny = batching(tokenizer_tiny,model_tiny)
batch_mini = batching(tokenizer_mini,model_mini)
batch_small = batching(tokenizer_small,model_small)
batch_medium = batching(tokenizer_medium,model_medium)
predicted_labels_tiny = np.concatenate((batch_tiny))
predicted_labels_mini = np.concatenate((batch_mini))
predicted_labels_small = np.concatenate((batch_small))
predicted_labels_medium = np.concatenate((batch_medium))

In [38]:
actual_labels= test_data.LABEL_COLUMN.values.tolist()

In [39]:
from sklearn import metrics
print("Tiny accuracy: ", metrics.accuracy_score(actual_labels, predicted_labels_tiny))
print("Mini accuracy: ", metrics.accuracy_score(actual_labels, predicted_labels_mini))
print("Small accuracy: ", metrics.accuracy_score(actual_labels, predicted_labels_small))
print("Medium accuracy: ", metrics.accuracy_score(actual_labels, predicted_labels_medium))

Tiny accuracy:  0.80496
Mini accuracy:  0.83812
Small accuracy:  0.85288
Medium accuracy:  0.85492


In [40]:
print("Tiny Report: ")
pd.DataFrame(metrics.classification_report(actual_labels, predicted_labels_tiny, output_dict=True)).T

Tiny Report: 


,precision,recall,f1-score,support
0,0.828112,0.76968,0.797827,12500.00000
1,0.784860,0.84024,0.811607,12500.00000
accuracy,0.804960,0.80496,0.804960,0.80496
macro avg,0.806486,0.80496,0.804717,25000.00000
weighted avg,0.806486,0.80496,0.804717,25000.00000


In [41]:
print("Mini Report: ")
pd.DataFrame(metrics.classification_report(actual_labels, predicted_labels_mini, output_dict=True)).T

Mini Report: 


,precision,recall,f1-score,support
0,0.837284,0.83936,0.838320,12500.00000
1,0.838961,0.83688,0.837919,12500.00000
accuracy,0.838120,0.83812,0.838120,0.83812
macro avg,0.838122,0.83812,0.838120,25000.00000
weighted avg,0.838122,0.83812,0.838120,25000.00000


In [42]:
print("Small Report: ")
pd.DataFrame(metrics.classification_report(actual_labels, predicted_labels_small, output_dict=True)).T

Small Report: 


,precision,recall,f1-score,support
0,0.828640,0.88976,0.858113,12500.00000
1,0.880981,0.81600,0.847246,12500.00000
accuracy,0.852880,0.85288,0.852880,0.85288
macro avg,0.854810,0.85288,0.852680,25000.00000
weighted avg,0.854810,0.85288,0.852680,25000.00000


In [43]:
print("Medium Report: ")
pd.DataFrame(metrics.classification_report(actual_labels, predicted_labels_medium, output_dict=True)).T

Medium Report: 


,precision,recall,f1-score,support
0,0.815698,0.91704,0.863405,12500.00000
1,0.905271,0.79280,0.845311,12500.00000
accuracy,0.854920,0.85492,0.854920,0.85492
macro avg,0.860484,0.85492,0.854358,25000.00000
weighted avg,0.860484,0.85492,0.854358,25000.00000
